# Einführung in Machine Learning

## Was ist Machine Learning?

Mithilfe von Machine Learning (maschinelles Lernen) werden IT-Systeme in die Lage versetzt, auf Basis vorhandener Daten und Algorithmen Muster und Gesetzmäßigkeiten zu erkennen. Die aus diesen Daten, den sog. Trainingsdaten, gewonnenen Erkenntnisse lassen sich für die Analyse von bisher unbekannten Daten anwenden, wie zum Beispiel

* wird dieses Gerät (z.B. ein Fahrstuhl, , eine Turbine, ein Windrad etc.) demnächst ausfallen?
* unter welcher Krankheit leidet dieser Patient?
* liegt ein Kreditkartenbetrug vor?
* welches Marketingmodell kann ich auf bestimmte Personen anwenden?
* wie viele Autos werden an einem bestimmten Datum auf einer Straße erwartet?
* Wird ein Kunde einen bestimmten Artikel kaufen?
* welches Objekt ist auf diesem Foto zu sehen?
* u.s.w.

Machine Learning wird häufig im Zusammenhang mit Künstlicher Intelligenz erwähnt. Der Begriff Intelligenz trifft allerdings in Bezug auf Machine-Learning nicht ganz zu, da es weniger um Intelligenz, sondern eher um durch Maschinen (gemeint sind Computer) erkennbare Muster geht.

In diesem Skript werden anhand eines einfachen Beispiels einige typische Verfahren im Bereich Machine Learning vorgestellt, ohne auf die mathematischen Hintergründe einzugehen. Vor dem Erstellen eines Prognose-Modells werden wir einige Diagramme erstellen. Dies ist durchaus üblich, um vorab die Daten besser zu verstehen (sog. EDA - Explorative
Datenanalyse).

Für die Erstellung von Diagrammen und der Machine Learning Modelle werden wir die Programmiersprache R verwenden. Allerdings geht es hier nicht um das Erlernen dieser Programmiersprache, sondern nur um die grundsätzlichen Möglichkeiten im Bereich Machine Learning. Neben R ist zum Beispiel auch noch die Sprache Python weit verbreitet. Es gibt auch grafische Tools, die für Machine Learning eingesetzt werden, wie z.B. das Programm Knime.

Exemplarisch wollen wir in diesem Skript die Daten einer Kundendatenbank eines Fahrradhändlers auswerten, um mit Hilfe eines Vorhersagemodells zu prognostizieren, wie viel Geld ein Bestandskunde voraussichtlich pro Monat für Fahrradzubehör ausgeben wird und ob ein Kunde ein Fahrrad kaufen wird oder nicht. Dies ist für entsprechende Marketingmaßnahmen ein sehr hilfreiches Instrument!

Doch zuerst wollen wir uns einen Überblick über den bestehenden Datensatz verschaffen. Dazu laden wir zuerst den Datensatz aus der Datei (BikeCustomers.csv) in den speicher und geben exemplarisch die ersten 6 Zeilen des Datensatzes aus:

In [ ]:
# Zuerst müssen wir einige Packages laden, die wir in diesem Skript verwenden wollen
if(!require("lubridate")) install.packages("lubridate", quietly = T)
if(!require("dplyr")) install.packages("dplyr", quietly = T)
if(!require("ggplot2")) install.packages("ggplot2", quietly = T)
if(!require("rio")) install.packages("caret", quietly = T)
if(!require("scales")) install.packages("scales", quietly = T)
if(!require("rpart")) install.packages("rpart", quietly = T)
if(!require("rpart.plot")) install.packages("rpart.plot", quietly = T)
if(!require("caret")) install.packages("caret", quietly = T)

print("Alle Packages geladen!")

In [ ]:
# Datensatz laden
daten <- import("BikeCustomers.csv")
# Mit head heben wir die ersten 6 Zeilen des Datensatzes aus
head(daten)

Die Spalten der Tabelle (die sog. Features) haben dabei folgende Bedeutung:


|Feature | Bedeutung |
|--------|-----------|
|CustomerID (integer)| Eindeutige Nummer des Kunden|
|Title (string)| Anrede (Mr, Mrs, Ms, Miss Dr, etc.)|
|FirstName (string)| Vorname|
|MiddleName (string)| Zweiter Vorname|
|LastName (string)| Nachname|
|Suffix (string)| Namenszusatz (Jr, Sr, etc.)|
|AddressLine1 (string)| Adresse1|
|AddressLine2 (string)| Adresse2|
|City (string)| Wohnort des Kunden|
|StateProvince (string)| Bundesstaat, in dem der Kunde lebt|
|CountryRegion (string)| Land oder Region, in dem der Kunde lebt|
|PostalCode (string)| Postleitzahl des Wohnortes|
|PhoneNumber (string)| Telefonnummer des Kunden|
|BirthDate (date)| Geburtsdatum im Format JJJJ-MM-TT|
|Education (string)| Höchster Bildungsabschluss des Kunden|
|Occupation (string)| Bereich, in dem der Kunde beruflich tätig ist|
|Gender (string)| Geschlecht|
|MaritalStatus (string)| Familienstand|
|HomeOwnerFlag (integer)| Ob der Kunde in einem Eigenheim lebt (1) oder nicht (0)|
|NumberCarsOwned (integer)| Anzahl der Autos im Haushalt|
|NumberChildrenAtHome (integer)| Anzahl der Kinder im Haushalt|
|TotalChildren (integer)| Anzahl der Kinder insgesamt|
|YearlyIncome (decimal)| Jährliches Bruttoeinkommen|
|AveMonthSpend (decimal)| Durchschnittliche Ausgaben für Fahrrad(-Zubehör) im Monat|
|BikeBuyer (integer)| Ob  der Kunde  bereits ein Fahrrad gekauft hat (1) oder nicht (0)|


Wir müssen vorab die Daten noch etwas anpassen. So sind die Geburtsdaten aktuell als Zeichenkette hinterlegt (sog. String). Wir müssen diese in Daten vom Typ *Date* umwandeln. Außerdem haben wir noch sog. kategoriale Variablen vorliegen. Dies sind Daten, die nur bestimmte Werte annehmen können, so wie z.B. Geschlecht (Gender) oder die Berufsgruppe (Occupation). Diese Features können nur vorab definierte Werte (Kategorien) enthalten. Außerdem wollen wir noch zwei weitere Features hinzufügen: Aus dem aktuellen Datum und dem Geburtsdatum generieren wir die Spalte *Age*, außerdem wollen wir die Kunden noch in Altersgruppen einteilen (*Agegroup*):

|Altersgruppe|Codierung|
|:----------:|:-------:|
| <20        |    1    |
| 21 - 30    |    2    |
| 31 - 44    |    3    |
| 45 - 59    |    4    |
| >=60       |    5    |

In [ ]:
# Features anpassen
daten$BirthDate <- as.Date(daten$BirthDate, format = "%Y-%m-%d")
daten$Education <- factor(daten$Education)
daten$Gender <- factor(daten$Gender)
daten$MaritalStatus <- factor(daten$MaritalStatus)
daten$Occupation <- factor(daten$Occupation)
daten$HomeOwnerFlag <- factor(daten$HomeOwnerFlag)
daten$BikeBuyer <- ifelse(daten$BikeBuyer==1, "Yes", "No")
daten$BikeBuyer <- factor(daten$BikeBuyer)

# Geburtsdatum in Alter umwandeln und in Spalte Age speichern
daten$Age <- round( (year(today()) - year(daten$BirthDate)))
daten$BirthDate <- NULL # Birthdate brauchen wir nicht mehr

# Wir fügen noch eine Spalte Altersgruppe hinzu:
# <20, 20 - 30, 31-45, 46-60, >60

daten$Agegroup <- ifelse(daten$Age<20, 1, ifelse(daten$Age <31, 2, 
                                           ifelse(daten$Age<45, 3,
                                              ifelse(daten$Age<60, 4, 5))))
# Auch Agegroup ist kategorial
daten$Agegroup <- factor(daten$Agegroup) 

Der Datensatz ist nun für weitere Analysen vorbereitet. Geben wir zur Übersicht mal aus, aus wie vielen Zeilen und Spalten unser Datensatz nun besteht.

In [ ]:
# Anzahl Kunden (Zeilen) und Features (Spalten) insgesamt
dim(daten)

Uns liegen also die Daten von 17.209 Kunden vor. Geben wir eine Tabelle aus, die uns Informationen darüber liefert, wie viele dieser Kunden bereits ein Fahrrad gekauft haben und wie viele nicht:

In [ ]:
# Anzahl Kunden, die ein Fahrrad gekauft (Yes) bzw. nicht
# gekauft haben (No)
table(daten$BikeBuyer)

Erstellen wir nun ein paar Diagramme, um ein Gefühl für unseren Datensatz zu entwickeln. Dieser Vorgang wird auch häufig als EDA bezeichnet (Explorative DatenAnalyse).

Unabhängig von den Fahrrädern wäre es interessant zu wissen, wie hoch das durchschnittliche Jahreseinkommen pro Berufsgruppe ist:

In [ ]:
# Jährliches Einkommen nach Berufsgruppe
daten %>% ggplot(aes(Occupation, YearlyIncome)) + geom_bar(stat="identity") +
  scale_y_continuous(labels=format_format(scientific=FALSE,
                                          decimal.mark=".", big.mark=","))

Noch ein weiteres Balkendiagramm: Einkommen in Abhängigkeit von der Schulbildung:

In [ ]:
# Jährliches Einkommen nach Bildung
daten %>% ggplot(aes(Education, YearlyIncome)) + geom_bar(stat="identity") +
  scale_y_continuous(labels=format_format(scientific=FALSE,
                                          decimal.mark=".", big.mark=","))

Gibt es einen Zusammenhang zwischen Anzahl der Kinder und der Schulbildung? Ausgegeben als Tabelle und als Diagramm:

In [ ]:
daten %>% group_by(Education) %>% summarise(mean(TotalChildren))

daten %>% group_by(Education) %>%
  summarise(DurchschnittKinder = mean(TotalChildren)) %>%
  ggplot(aes(Education, DurchschnittKinder)) +
  geom_bar(stat = "identity")

Wir geben das Jahreseinkommen in Abhängigkeit der Bildung als Boxplot-Diagramm aus.

In [ ]:
daten %>% ggplot(aes(Education, YearlyIncome)) +
  geom_boxplot(fill="red")

Unterscheidet sich das Einkommen zwischen den Kunden, die über Wohneigentum verfügen oder nicht? Wir geben das wieder als Boxplot aus.

In [ ]:
daten %>% ggplot(aes(HomeOwnerFlag, YearlyIncome)) +
  geom_boxplot(fill="red") +
  scale_y_continuous(labels=format_format(scientific=FALSE,
                                          decimal.mark=".",
                                          big.mark=","))

Wie unterscheidet sich das Einkommen zwischen den definierten Altersgruppen?

In [ ]:
# Einkommen bzgl. Altersgruppe
daten %>% ggplot(aes(Agegroup, YearlyIncome)) +
  geom_boxplot(fill="red") +
  scale_y_continuous(labels=format_format(scientific=FALSE,
                                          decimal.mark=".",
                                          big.mark=",")) +
  ggtitle("Einkommen je Altersgruppe") +
  xlab("Altersgruppe") + ylab("Jahreseinkommen")

Wie hängen die durchschnittlichen Ausgaben pro Monat für Fahrräder vom Einkommen ab? Wir geben jeden einzelnen Wert für *AveMonthSpend* als Punkt aus (sog. Scatterplot). Wir färben die Punkte nach der beruflihen Tätigkeit ein. Zusätzlich fügen wir eine Trendlinie ein, eine sog. Regressionsgerade.

In [ ]:
# Durchschn. Ausgaben pro Monat in Abhängigkeit vom Einkommen
daten %>% ggplot(aes(YearlyIncome, AveMonthSpend, color=Occupation)) +
  geom_point(alpha=.3) +
  geom_smooth(method="lm", color="red", size=2)

Bisher haben wir nur Diagramme bestehend auf den vorliegenden Daten erstellt. Jetzt wollen wir aber ein Beispiel für Machine Learning durchführen: Im zuletzt generierten Diagramm haben wir eine Trendlinie erstellt, um den Wert für *AveMonthSpend* anhand einer weiteren Information (YearlyIncome) einen Wert für AveMonthSpend vorherzusagen. Wir wollen nun doch weitere Informationen einfließen lassen, um die Vorhersagegenauigkeit zu erhöhen.

Die Vorhersage soll auf Basis der Features *Agegroup*, *Education*, *Occupation*, *HomeOwnerFlag*, *NumberCarsOwned*, *MaritalStatus* und *Gender* erfolgen. Dazu erstellen wir ein Vorhersagemodell und können dann eine Prognose wagen. Um die QUalität unseres Modells zu bestimmen, teilen wir unseren Datensatz in einen Trainings- und Testdatensatz auf. Mit dem Trainingsdatensatz erstellen ("trainieren") wir das Modell. Mit dem Testdatensatz vergleichen wir unsere Vorhersage (Prediction) mit dem realen Wert aus dem Testdatensatz. Der nachfolgende Quellcode führt alle diese Schritte aus und gibt zufällig 20 Zeilen aus, die sowohl unsere Vorhersage als auch den wahren Wert für *AveMonthSpend* anzeigen:

In [ ]:
# Vorhersage, wieviel ein Kunde monatlich für Fahrräder
# ausgibt (AvgMonthSpend).
# Bekannt sind: YearlyIncome, Agegroup, Education, MaritalStatus, Gender
set.seed(42)

# Aufteilen in Trainings- und Testdatensatz
index <- createDataPartition(daten$AveMonthSpend, p=.8, list=FALSE)
train <- daten[index, ]    # Trainingsdaten
test  <- daten[-index, ]   # Testdaten


# Unser Vorhersagemodell erstellen
modell <- lm(AveMonthSpend~Agegroup+Education+Occupation+
           HomeOwnerFlag+NumberCarsOwned+NumberChildrenAtHome+
           TotalChildren+Age+Agegroup+Gender, data=train )

# Vorhersage (Prediction) anhand des Modells erstellen
pred <- predict(modell, test, type="response")

# Gib zufällig Vergleichszahlen aus (Real vs. Pred)
Ergebnisse <- data.frame(Vorhersage=round(pred), 
                         RealerWert=test$AveMonthSpend)
sample_n(Ergebnisse, 20)

## Bestimmung der Qualität eines Modells
Wir stellen fest, dass unser Modell eine recht gute Vorhersage liefert!

Wir wollen noch ein weiteres Modell erstellen! Interessant ist sicher auch, ob Personen, die über bestimmte Merkmale verfügen, eher ein Fahrrad kaufen werden oder nicht. Bei Personen, die mit höherer Wahrscheinlichkeit ein Fahrrad kaufen werden, können wir gezielt im Rahmen einer Marketing-Kampagne ansprechen, z.B. durch Sonderrabatte.



Wir wollen die Daten der Merkmale *Gender*, *YearlyIncome*, *Agegroup*, *Occupation* und *NumberChildrenAtHome* in unser Modell einfließen lassen. Als Modell verwenden wir einen sog. "Entscheidungsbaum" (Decision Tree), das das Modell eine Kategorie vorhersagen soll: Wird der Kunde ein Fahrrad kaufen oder nicht? Der folgende R-Code erstellt das Modell und gibt auch den erstellten Entscheidungsbaum grafisch aus.

In [ ]:
# Vorhersage, ob ein Kunde ein Fahrrad kaufen wird
# mit Hilfe eines Entscheidungsbaums (Decision Tree)

rpcontrol <- rpart.control(minsplit = 2, minbucket = 3, cp=.003)
modell2 <- rpart(BikeBuyer~Gender+YearlyIncome+Agegroup+Occupation+
                   NumberChildrenAtHome, data=train,
                 control=rpcontrol)

rpart.plot(modell2, cex=1, branch.col=10, branch.lwd=4, type=1)

Wie gut war nun unser Vorhersagemodell? Wir geben dazu eine Tabelle aus, die die Anzahl der richtig, aber auch falsch zugeordneten Kunden gegenüberstellt:

In [ ]:
# Testen unseres Modells

# Vorhersage (Prediction) anhand des Modells
pred2 <- predict(modell2, test, type = "class")

# Gegenüberstellung der Daten
table(pred2, test$BikeBuyer)

Wie gut ist nun das Modell? Eine Statistik, die das in einer Zahl zusammenfasst, nennt sich *Accuracy*. Das ist das Verhältnis der richtig vorhergesagten Daten zur Gesamtzahl der Beobachtungen. Je näher der Wert bei 1,0 liegt, desto besser. In unserem Beispiel beträgt die *Accuracy* = (2117+592) / (2117+592+227+504) = 0,79:

In [ ]:
# Accuracy ausgeben
confusionMatrix(pred2, test$BikeBuyer)$overall["Accuracy"]

Unser Modell lag also bei knapp 80% der Daten richtig, was kein schlechter Wert ist!